In [2]:
import cv2
import torch
from PIL import Image

# Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model = torch.hub.load('yolov5-master', 'custom', path='best.pt', source='local') 


img = Image.open('test/bike2.jpg')

# Inference
results = model(img, size=370)  # includes NMS

# Results
results.print()  
results.show()
# results.show()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0] 

F:\Anaconda\envs\pytorch_env\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'F:\Anaconda\envs\pytorch_env\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
YOLOv5  2022-2-12 torch 1.10.2 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3072MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 
image 1/1: 600x960 3 Cyclists
Speed: 13.0ms pre-process, 464.7ms inference, 6.0ms NMS per image at shape (1, 3, 256, 384)


,xmin,ymin,xmax,ymax,confidence,class,name
0,5.858765,3.705006,438.86499,600.000000,0.805725,0,Cyclist
1,624.528442,28.872852,813.14093,588.162109,0.572550,0,Cyclist
2,416.936188,8.454857,621.17395,544.063171,0.459286,0,Cyclist


In [23]:
def draw_text(img, text,
          pos,
          font=cv2.FONT_HERSHEY_PLAIN,
          font_scale=1,
          font_thickness=1,
          text_color=(255, 255, 255),
          text_color_bg=(255, 0, 0)
          ):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)

    return text_size

In [29]:

input_video_path = 'test/bikes.mp4'
output_video_path = 'bike_result_small_yolo.mp4'

cap = cv2.VideoCapture(input_video_path)


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # float `width`
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(width, height, fps)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    resized_frame = cv2.resize(frame, (370, 370), interpolation = cv2.INTER_AREA)
    results = model(resized_frame, size=370)

    for index, row in results.pandas().xyxy[0].iterrows():
        scale_x = width/370
        scale_y = height/370
        left  = int(row.xmin*scale_x)
        top = int(row.ymin*scale_y)
        right  = int(row.xmax*scale_x)
        bottom = int(row.ymax*scale_y)

        cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
        draw_text(frame, f"{row['name']} {round(row.confidence, 2)}", (left, top))
    out.write(frame)
    cv2.imshow('frame', frame)
    c = cv2.waitKey(1)
    if c & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

1280 720 29.97002997002997
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
Empty DataFrame
Columns: [xmin, ymin, xmax, ymax, confidence, class, name]
Index: []
         xmin        ymin        xmax        ymax  confidence  class     name
0  179.430450  119.124718  234.816467  304.853699    0.576079      0  Cyclist
1  102.835327  185.300674  121.750061  237.448013    0.561207      0  Cyclist
         xmin        ymin        xmax        ymax  confidence  class     name
0  184.545441  108.161926  238.270081  308.713257    0.489469     

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
